In [25]:
import torch
import torch.nn as nn
import re
import numpy as np
import json
import os
# from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader,TensorDataset
from torch.utils.tensorboard import SummaryWriter 
# from google.colab import drive
# drive.mount('/content/drive')



In [26]:
def split_get_label_from_filename(filename):
    return label_mapping.get(filename[8:-5],-1)

def get_label_from_filename(filename):
    match = re.search(r'_(\d+_?\d*)\.json$', filename)
    print(match)
    if match:
        label_key = match.group(1)
        return label_mapping.get(label_key, -1)  # 如果标签不匹配返回 -1

# 获取文件夹中的文件及其标签
def load_files_and_labels(folder_path,split=False):
    file_list = []
    labels = []
    for file in os.listdir(folder_path):
        # print("file: ", file)
        if file.endswith(".json"):
            if split:
                label = split_get_label_from_filename(file)
            else: label = get_label_from_filename(file)
            if label != -1:
                file_list.append(os.path.join(folder_path, file))
                labels.append(label)
            #print(f"File: {file}, Label: {label}")

    return file_list, labels




In [27]:
import json
import numpy as np

# 1. 提取字符并编码为整数索引
def hex_to_sequence(hex_feature):
    """
    将十六进制字符串转换为整数索引序列。
    去掉冒号并转换为字符对应的索引。
    """
    hex_chars = hex_feature.replace(":", "")
    char_to_index = {char: idx for idx, char in enumerate("0123456789abcdef")}
    return [char_to_index[char] for char in hex_chars]

def process_json(file_path):
    """
    处理 JSON 文件，提取时间戳、包长度和 raw data 特征，并对特征进行归一化。
    """
    features = []
    try:
        # 打开 JSON 文件
        with open(file_path, 'r', encoding='utf-8') as file:
            datas = json.load(file)

        # 遍历 JSON 数据，提取时间戳和包长度
        initial_timestamp = None
        for data in datas:
            try:
                timestamp = float(data["_source"]["layers"]["frame"]["frame.time_relative"])  # 时间戳
                packet_length = int(data["_source"]["layers"]["frame"]["frame.len"])  # 包长度

                # 获取数据部分，若不存在则为0
                if 'data' in data["_source"]["layers"]:
                    rawdata = data["_source"]["layers"]["data"]["data.data"]
                else:
                    rawdata = '0'

                # 将原始数据转换为整数序列
                data_feature = hex_to_sequence(rawdata)

                # 将数据填充或截断为指定长度 2832
                if len(data_feature) < 2832:  # 如果长度小于 2832，填充 0
                    data_feature += [0] * (2832 - len(data_feature))
                elif len(data_feature) > 2832:  # 如果长度大于 2832，截断
                    data_feature = data_feature[:2832]

                # 如果数据长度不符合预期，打印出来
                if len(data_feature) != 2832:
                    print(f"Data feature length mismatch: {len(data_feature)}")

                # 初始化时间戳
                if initial_timestamp is None:
                    initial_timestamp = timestamp  # 设置初始时间戳

                # 计算相对时间戳
                relative_timestamp = timestamp - initial_timestamp
                # print(type(relative_timestamp))


                timestamp_array = np.array([relative_timestamp], dtype=float)
                packet_length_array = np.array([packet_length], dtype=float)/1512
                data_feature = np.array(data_feature, dtype=float)/15
           
                # 将特征按顺序组合为 [时间戳, 包长度, 数据特征]
                feature = np.hstack((timestamp_array, packet_length_array,data_feature))
                
                # 添加到特征列表
                features.append(feature)

            except (KeyError, ValueError) as e:
                # 跳过有问题的数据包
                print(f"Skipping packet due to error: {e}")
                continue

        features_array=np.array(features)
        max_timestamp = np.max(features_array[:, 0])  # 获取最大时间戳
        # print("max")
        features_array[:, 0] = [feature[0] / max_timestamp for feature in features_array]  # 时间戳归一化
        # print("11") 

    except Exception as e:
        print(Exception)
        print(f"Error processing file {file_path}: {e}")
        features_array = np.zeros((1, 2834))  # 返回空特征以避免程序中断

    return features_array

In [4]:
# file_path = "202_packet_json_new/b1_p1_0_2.json"
# features = process_json(file_path)
# #print(features)
# print(features[0])

# print(f"Feature shape: {features.shape}")
# print(f"Feature type: {type(features)}")
# print(features[:][2].shape)

In [28]:
import nest_asyncio
nest_asyncio.apply()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 定义文件夹路径 /content/drive/MyDrive/202_project/202_packet_json

split = False
# folder_path = "packet_json_split"
if split:
    folder_path = "packet_json_split" 
else:
    folder_path = "202_packet_json_new"

# 定义标签映射
label_mapping = {
    "0": 0,             # static
    "0_2": 1,           # slightly_move
    "1": 2,             # move
    "4": 3              # intensely_move
}

# 调用函数获取文件和标签
data_paths, labels = load_files_and_labels(folder_path,split)
# 用于存储每个样本的特征

print(type(labels))
labels_array=np.array(labels)#[:50]
print(labels_array.shape)
print(labels_array)
labels_tensor=torch.tensor(labels_array,dtype=torch.long)

Using device: cuda
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 12), match='_0.json'>
<re.Match object; span=(5, 12), match='_0.json'>
<re.Match object; span=(5, 12), match='_1.json'>
<re.Match object; span=(5, 14), match='_0_2.json'>
<re.Match object; span=(5, 12), match='_4.json'>
<re.Match object; span=(5, 12), match='_

In [6]:
features_list = []

# 遍历每个样本并生成特征


       # 将特征添加到列表中

# 查看生成的 features_list

# count = 0

for data_path in data_paths:
  # count +=1
  print(data_path)
  features = process_json(data_path)
  features_list.append(features)
  # if count >=50: break

  print(features.shape)
print(f"生成了 {len(features_list)} 个样本特征")
print(f"第一个样本的特征形状: {features_list[0].shape}")
#  print(data_path)
  #print(labels[i])

202_packet_json_new/b3_p5_0_2.json
(13948, 2834)
202_packet_json_new/b1_p5_1.json
(11166, 2834)
202_packet_json_new/b4_p4_1.json
(12725, 2834)
202_packet_json_new/b3_p1_1.json
(13638, 2834)
202_packet_json_new/b1_p1_4.json
(12107, 2834)
202_packet_json_new/b1_p3_4.json
(11231, 2834)
202_packet_json_new/b3_p1_4.json
(13053, 2834)
202_packet_json_new/b5_p1_1.json
(12979, 2834)
202_packet_json_new/b5_p1_0_2.json
(12855, 2834)
202_packet_json_new/b2_p1_0_2.json
(13799, 2834)
202_packet_json_new/b4_p3_0_2.json
(12835, 2834)
202_packet_json_new/b4_p1_4.json
(13320, 2834)
202_packet_json_new/b3_p3_1.json
(13068, 2834)
202_packet_json_new/b1_p2_1.json
(10835, 2834)
202_packet_json_new/b2_p4_0.json
(14132, 2834)
202_packet_json_new/b1_p3_0.json
(11783, 2834)
202_packet_json_new/b2_p4_1.json
(13414, 2834)
202_packet_json_new/b5_p3_0_2.json
(12195, 2834)
202_packet_json_new/b4_p5_4.json
(12929, 2834)
202_packet_json_new/b5_p3_0.json
(12413, 2834)
202_packet_json_new/b3_p4_4.json
(12968, 2834)
202

In [29]:
split=False
if split:fixed_time_steps = 3000
else: fixed_time_steps = 12000
# 截断或补零到固定长度
aligned_features = []
for feature in features_list:
    if feature.shape[0] > fixed_time_steps:
        truncated = feature[:fixed_time_steps, :]  # 截断
    else:
        truncated = np.pad(feature, ((0, fixed_time_steps - feature.shape[0]), (0, 0)), mode='constant')  # 补零
    aligned_features.append(truncated)

# 转为张量
features_tensor =  torch.tensor(np.stack(aligned_features, axis=0),dtype=torch.float32)
print(f"截断后的特征张量形状: {features_tensor.shape}")
print(type(features_tensor))

截断后的特征张量形状: torch.Size([100, 12000, 2834])
<class 'torch.Tensor'>


In [30]:
if split:
    train_dataset=TensorDataset(features_tensor[0:210],labels_tensor[0:210])
    eval_dataset =TensorDataset(features_tensor[210:-1],labels_tensor[210:-1])
else:
    train_dataset=TensorDataset(features_tensor[0:70],labels_tensor[0:70])
    eval_dataset =TensorDataset(features_tensor[70:-1],labels_tensor[70:-1])
batch_size=16
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
eval_loader  = DataLoader(eval_dataset,batch_size=batch_size,shuffle=True)

    
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=2,dropout=0.2):
        super(LSTMModel, self).__init__()
        
        # Define an LSTM with multiple layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers, batch_first=True,dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # Forward pass through LSTM
        out, (hn, cn) = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last time-step's output for classification
        return out


In [31]:
input_size=2834
hidden_size=128
output_size=4
num_layers = 1     # number of LSTM layers
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model.to(device)
model = LSTMModel(input_size, hidden_size, output_size, num_layers=num_layers,dropout=0.4)
# model.add(Dense(units))

# inputs=features_tensor
model = model.to(device)
# model = model
# inputs = inputs.to(device)
# labels = labels_tensor
torch.cuda.empty_cache()
# outputs = model(inputs)

# print(f"outputs:{outputs.shape}")

In [32]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))


True
0
NVIDIA GeForce RTX 4090


In [ ]:

# 5. 损失函数和优化器
import  numpy 
import numpy
print(numpy.__version__)

criterion = nn.CrossEntropyLoss()  # 分类任务使用交叉熵损失
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)  # Decay learning rate every 10 epochs
# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)  # Decrease LR by 5% every epoch
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.95,patience=5)
# pretrain=True
# model_path="model_epoch_21tran_acc_88.57% val_acc_48.28%.pth"
# if pretrain:
#     model.load_state_dict(torch.load(model_path))
#     model.to(device)
writer = SummaryWriter('runs/lstm_training')  # This will store logs in 'runs/lstm_training'

# 6. 训练循环
num_epochs = 1000  # 训练轮次
max_accu=0.25

model.eval()
num=0
# Disable gradient computation
with torch.no_grad():
    test_preds = []  # To store predictions
    test_labels = []  # To store true labels
    
    # Loop through the train_loader (or test_loader if you're evaluating the test set)
    for inputs, targets in eval_loader:
        # Move inputs and targets to the same device as the model
        inputs = inputs.to(device)
        targets = targets.to(device)

        # Ensure the input shape is (batch_size, seq_len, input_size) for LSTM
        # Adjust this depending on the actual input shape
        # inputs = inputs.view(inputs.size(0), -1, inputs.size(1))  # Assuming inputs have shape (batch_size, input_size)
        
        # Forward pass through the model
        outputs = model(inputs)
        
        # Get predictions by taking the argmax along the output dimension
        _, predicted = torch.max(outputs, 1)
        num+=1
        
        # Store predictions and true labels as NumPy arrays
        # Store predictions and true labels as NumPy arrays
        test_preds.extend(predicted.cpu().numpy())
        test_labels.extend(targets.cpu().numpy())
        
        # Print predictions and labels for debugging (optional)
    print("Predictions: ", test_preds)
    print("True Labels: ", test_labels)

    # You may want to calculate accuracy or other metrics after the loop
    # For example, calculate accuracy:

    accuracy = sum(np.array(test_preds) == np.array(test_labels)) / len(test_labels)
    # print(f"Accuracy: {accuracy * 100:.2f}%")
    diff_indices = [i for i in range(len(test_labels)) if test_labels[i] != test_preds[i]]

    print(diff_indices)
    


    # accuracy = accuracy_score(test_labels, test_preds)
    print(f"Test Accuracy: {accuracy*100:.2f}%")




2.0.2
Predictions:  tensor([3, 1, 3, 0, 0, 3, 3, 0, 2, 0, 3, 2, 2], device='cuda:0')
True Labels:  tensor([0, 0, 1, 1, 2, 2, 0, 1, 2, 2, 2, 3, 1], device='cuda:0')


NameError: name 'diff_indices' is not defined

In [ ]:
# print(data_paths(diff_indices))

diff_datapaths = [data_paths[i] for i in diff_indices]
for index in diff_indices:
    print(f"Index: {index}, Datapath: {data_paths[index]}")
print(diff_datapaths)
